In [ ]:
# Урок 10. Обучение без учителя в Scikit-learn. Видеоурок


In [ ]:
# Задание 1
# Импортируйте библиотеки pandas, numpy и matplotlib.
# Загрузите "Boston House Prices dataset" из встроенных наборов данных библиотеки sklearn. 
# Создайте датафреймы X и y из этих данных.

In [ ]:
# Разбейте эти датафреймы на тренировочные (X_train, y_train) и тестовые (X_test, y_test)
# с помощью функции train_test_split так, чтобы размер тестовой выборки 
# составлял 20% от всех данных, при этом аргумент random_state должен быть равен 42.

In [ ]:
# Масштабируйте данные с помощью StandardScaler.

In [ ]:
# Постройте модель TSNE на тренировочный данных с параметрами:
# n_components=2, learning_rate=250, random_state=42.

In [ ]:
# Постройте диаграмму рассеяния на этих данных.

In [ ]:
# Задание 2
# С помощью KMeans разбейте данные из тренировочного набора на 3 кластера, 
# используйте все признаки из датафрейма X_train.
# Параметр max_iter должен быть равен 100, random_state сделайте равным 42.

In [ ]:
# Постройте еще раз диаграмму рассеяния на данных, полученных с помощью TSNE,
# и раскрасьте точки из разных кластеров разными цветами.

In [ ]:
# Вычислите средние значения price и CRIM в разных кластерах.

In [ ]:
# * Задание 3
# Примените модель KMeans, построенную в предыдущем задании,
# к данным из тестового набора.

In [ ]:
# Вычислите средние значения price и CRIM в разных кластерах на тестовых данных.
